In [42]:
"""本代码完成由以下两部分组成（两个def）：
1.通过tushare获取最新的若干指数的成份股的构成(tusahre接口需要自备)
2.爬取最新的若干指数的成份股的看涨看跌数据，并保存到本地excel，单个sheet代表了单个指数下各个股票的看涨看跌情况"""
import pandas as pd
import numpy as np
import requests
import re
from tqdm import tqdm
import selenium
import pymysql
from datetime import datetime
import schedule
import time
import tushare as ts
import os

base_url='https://eminterservice.eastmoney.com/UserData/GetWebTape?code='
headers={
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 SLBrowser/9.0.3.5211 SLBChan/25'
}
def components_history(index_file_path,tushare_token,index_tuple=['000016.SH','399300.sz','000905.SH','000852.SH','000985.SH']):

    df=pd.DataFrame(columns=['index_code','con_code','trade_date','weight'])
    for index in tqdm(index_tuple):
        pro=ts.pro_api('20240522230128-4d87f31a-4782-400e-8400-ee063ffe55a5')
        pro._DataApi__http_url = 'http://tsapi.majors.ltd:7000'

        tem_df = pro.index_weight(index_code=index)
        date=tem_df.iloc[0]['trade_date']
        df=pd.concat([df,tem_df[tem_df['trade_date']==date]],axis=0)

    df.to_csv(index_file_path)
    index_tuple=tuple(df['index_code'].unique())
    return df,index_tuple
index_tuple=('000016.SH','399300.sz','000905.SH','000852.SH','000985.SH')
df,index_tuple=components_history(r'D:\作业\实习工作相关\清萍量化\数据\指数成分.csv',
                      'd689cb3c1d8c8a618e49ca0bb64f4d6de2f70e28ab5f76a867b31ac7',
                      index_tuple=index_tuple)
df

100%|██████████| 5/5 [00:16<00:00,  3.22s/it]


,index_code,con_code,trade_date,weight
0,000016.SH,600519.SH,20240830,13.420
1,000016.SH,601318.SH,20240830,7.016
2,000016.SH,600036.SH,20240830,5.891
3,000016.SH,600900.SH,20240830,5.321
4,000016.SH,601899.SH,20240830,3.902
...,...,...,...,...
4935,000985.SH,871857.BJ,20240830,0.001
4936,000985.SH,831856.BJ,20240830,0.000
4937,000985.SH,832145.BJ,20240830,0.000
4938,000985.SH,838275.BJ,20240830,0.000


In [51]:


def web_tape(base_url, component_df, index_tuple=['000016.SH', '399300.sz', '000905.SH', '000852.SH', '000985.SH'],
             save_path=None, headers=None):
    # 创建一个ExcelWriter对象来保存到同一个Excel文件
    writer = pd.ExcelWriter(save_path, engine='xlsxwriter') if save_path else None

    for i in index_tuple:
        print(f"Processing index: {i}")
        df = pd.DataFrame(columns=['code', 'TapeZ', 'TapeD', 'TapeType', 'Date'])
        
        # 获取当前指数对应的股票代码
        code_tuple = tuple(component_df[component_df['index_code'] == i]['con_code'])
        
        for code in tqdm(code_tuple):
            # 修改代码字符串格式
            
            temcode = code[-2:].lower() + code[:-3]
            url = base_url + temcode

            # 请求数据
            r = requests.get(url, headers=headers)
            pattern = re.findall(r'"TapeZ":(.*?),"TapeD":(.*?),"TapeType":(.*?),"Date":"(.*?)"', r.text)
            if pattern:
                # 组织数据
                code_list = [code]
                code_list.extend(pattern[0])
                code_list=[code_list]
            else:
                print(f"指数{i}下面的{code}存在无法匹配情况，请检查")


            
            df = pd.concat([df, pd.DataFrame(code_list,columns=['code', 'TapeZ', 'TapeD', 'TapeType', 'Date'])], axis=0).reset_index(drop=True)
        
        # 将当前DataFrame写入到Excel的一个Sheet中
        if writer:
            df.to_excel(writer, sheet_name=i, index=False)
            #writer.close()    
    # 保存Excel文件
    if writer:
        writer.close()

    return df

base_url = 'https://eminterservice.eastmoney.com/UserData/GetWebTape?code='
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 SLBrowser/9.0.3.5211 SLBChan/25'
}

# 假设 df 是事先加载的DataFrame
tape_df = web_tape(base_url, df, index_tuple=['000016.SH', '399300.sz', '000905.SH', '000852.SH', '000985.SH'], 
                   save_path=r'D:\作业\实习工作相关\清萍量化\数据\涨跌看法数据.xlsx', headers=headers)

print(tape_df)


Processing index: 000016.SH


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:02<00:00, 17.19it/s]


Processing index: 399300.sz


0it [00:00, ?it/s]


...

Processing index: 000852.SH


100%|██████████| 1000/1000 [01:00<00:00, 16.53it/s]


Processing index: 000985.SH


100%|██████████| 4940/4940 [05:08<00:00, 16.04it/s]


           code   TapeZ   TapeD TapeType        Date
0     600519.SH  0.2047  0.7953        0  2024-09-07
1     300750.SZ  0.5445  0.4555        0  2024-09-07
2     601318.SH  0.5543  0.4457        0  2024-09-07
3     600036.SH  0.5387  0.4613        0  2024-09-07
4     600900.SH   0.459   0.541        0  2024-09-07
...         ...     ...     ...      ...         ...
4935  871857.BJ  0.3165  0.6835        0  2024-09-07
4936  831856.BJ  0.2582  0.7418        0  2024-09-07
4937  832145.BJ  0.2521  0.7479        0  2024-09-07
4938  838275.BJ  0.2032  0.7968        0  2024-09-07
4939  834765.BJ  0.2893  0.7107        0  2024-09-07

[4940 rows x 5 columns]
